In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter1d


# ---------- filename → patient_id ----------

def extract_patient_id(path: Path) -> str:
    """
    Extract patient ID from filename.
    Assumes the whole stem is the patient id, e.g. 'P01.csv' -> 'P01'.
    If you actually have filenames like 'P01_session1.csv',
    change this to:  return path.stem.split('_')[0]
    """
    return path.stem


# ---------- numeric helpers ----------

def resample_to_grid(df: pd.DataFrame,
                     value_col: str,
                     grid: pd.DatetimeIndex) -> pd.Series:
    """
    Reindex a single column onto the common grid and interpolate in time.
    Ensures numeric dtype.
    """
    df = df.sort_index()
    s = df[value_col].reindex(grid)

    # Ensure numeric dtype (strings -> NaN)
    s = pd.to_numeric(s, errors="coerce")

    # Time-based interpolation
    s_interp = s.interpolate(method="time", limit_direction="both")
    s_interp.name = value_col
    return s_interp


def gaussian_smooth(series: pd.Series, sigma: float = 2.0) -> pd.Series:
    """
    Apply 1D Gaussian smoothing to a time series.
    """
    numeric = pd.to_numeric(series, errors="coerce").astype(float)
    # Fill any remaining NaNs so SciPy is happy
    numeric = numeric.fillna(method="ffill").fillna(method="bfill")

    arr = numeric.to_numpy()
    smoothed = gaussian_filter1d(arr, sigma=sigma, mode="nearest")
    return pd.Series(smoothed, index=series.index, name=series.name + "_smooth")


def standardize(series: pd.Series, method: str = "zscore") -> pd.Series:
    """
    Standardize *within patient* (because we call this per patient).
    """
    x = pd.to_numeric(series, errors="coerce").astype(float).to_numpy()

    if method == "zscore":
        mu = np.nanmean(x)
        sigma = np.nanstd(x)
        if sigma == 0:
            standardized = np.zeros_like(x)
        else:
            standardized = (x - mu) / sigma
    elif method == "minmax":
        xmin = np.nanmin(x)
        xmax = np.nanmax(x)
        if xmax == xmin:
            standardized = np.zeros_like(x)
        else:
            standardized = (x - xmin) / (xmax - xmin)
    else:
        raise ValueError("method must be 'zscore' or 'minmax'")

    return pd.Series(standardized, index=series.index,
                     name=series.name + f"_{method}")


def build_common_grid(cgm: pd.DataFrame,
                      hr: pd.DataFrame,
                      freq: str = "1T") -> pd.DatetimeIndex:
    """
    Build a common regular time grid where both streams overlap.
    """
    start = max(cgm.index.min(), hr.index.min())
    end = min(cgm.index.max(), hr.index.max())
    if start >= end:
        raise ValueError("No overlapping time range between CGM and HR.")
    return pd.date_range(start=start, end=end, freq=freq)


# ---------- load all files with patient_id ----------

def load_all_cgm(cgm_dir: Path) -> pd.DataFrame:
    """
    Load all CGM CSVs and attach patient_id from filename.
    Returns a dataframe with columns: patient_id, time, glucose_level_mg_dl
    """
    dfs = []
    for csv_path in sorted(cgm_dir.glob("*.csv")):
        pid = extract_patient_id(csv_path)
        df = pd.read_csv(
            csv_path,
            parse_dates=["start_date_time", "end_date_time"],
            infer_datetime_format=True
        )

        # Midpoint of interval as timestamp
        df["time"] = df["start_date_time"] + (
            df["end_date_time"] - df["start_date_time"]
        ) / 2

        df["patient_id"] = pid
        dfs.append(df[["patient_id", "time", "glucose_level_mg_dl"]])

    if not dfs:
        raise ValueError(f"No CGM files found in {cgm_dir}")

    cgm_all = pd.concat(dfs, ignore_index=True)
    cgm_all = cgm_all.drop_duplicates(subset=["patient_id", "time"])
    return cgm_all


def load_all_hr(hr_dir: Path) -> pd.DataFrame:
    """
    Load all HR CSVs and attach patient_id from filename.
    Returns: patient_id, time, hr_bpm
    """
    dfs = []
    for csv_path in sorted(hr_dir.glob("*.csv")):
        pid = extract_patient_id(csv_path)
        df = pd.read_csv(
            csv_path,
            parse_dates=["date_time"],
            infer_datetime_format=True
        )
        df = df.rename(columns={"date_time": "time",
                                "heart_rate_beats_min": "hr_bpm"})
        df["patient_id"] = pid
        dfs.append(df[["patient_id", "time", "hr_bpm"]])

    if not dfs:
        raise ValueError(f"No HR files found in {hr_dir}")

    hr_all = pd.concat(dfs, ignore_index=True)
    hr_all = hr_all.drop_duplicates(subset=["patient_id", "time"])
    return hr_all


# ---------- per-patient preprocessing ----------

def preprocess_one_patient(pid: str,
                           cgm_all: pd.DataFrame,
                           hr_all: pd.DataFrame,
                           freq: str = "1T",
                           smoothing_sigma: float = 2.0,
                           standardize_method: str = "zscore") -> pd.DataFrame:
    """
    Preprocess CGM + HR for a single patient.
    """
    cgm_p = cgm_all[cgm_all["patient_id"] == pid].copy()
    hr_p = hr_all[hr_all["patient_id"] == pid].copy()

    if cgm_p.empty or hr_p.empty:
        raise ValueError(f"Missing data for patient {pid}")

    cgm_p = cgm_p.set_index("time")
    hr_p = hr_p.set_index("time")

    grid = build_common_grid(cgm_p, hr_p, freq=freq)

    glucose = resample_to_grid(cgm_p, "glucose_level_mg_dl", grid)
    hr = resample_to_grid(hr_p, "hr_bpm", grid)

    glucose_smooth = gaussian_smooth(glucose, sigma=smoothing_sigma)
    hr_smooth = gaussian_smooth(hr, sigma=smoothing_sigma)

    glucose_std = standardize(glucose_smooth, method=standardize_method)
    hr_std = standardize(hr_smooth, method=standardize_method)

    df = pd.concat(
        [glucose, hr, glucose_smooth, hr_smooth, glucose_std, hr_std],
        axis=1
    )
    df.insert(0, "patient_id", pid)   # make patient_id a column
    return df


# ---------- main: run for all patients ----------

def preprocess_all(root_dir: Path,
                   freq: str = "1T",
                   smoothing_sigma: float = 2.0,
                   standardize_method: str = "zscore") -> pd.DataFrame:
    cgm_dir = root_dir / "cgm_csv"
    hr_dir = root_dir / "heart_rate_csv"

    cgm_all = load_all_cgm(cgm_dir)
    hr_all = load_all_hr(hr_dir)

    # patients that have BOTH CGM and HR data
    patients_cgm = set(cgm_all["patient_id"].unique())
    patients_hr = set(hr_all["patient_id"].unique())
    patient_ids = sorted(patients_cgm & patients_hr)

    all_dfs = []
    for pid in patient_ids:
        try:
            print(f"Processing patient {pid}...")
            df_pid = preprocess_one_patient(
                pid, cgm_all, hr_all,
                freq=freq,
                smoothing_sigma=smoothing_sigma,
                standardize_method=standardize_method
            )
            all_dfs.append(df_pid)
        except ValueError as e:
            print(f"  Skipping {pid}: {e}")

    if not all_dfs:
        raise ValueError("No patients successfully processed.")

    final = pd.concat(all_dfs)
    # Save with time as a column for convenience
    final = final.reset_index().rename(columns={"index": "time"})
    out_path = root_dir / f"preprocessed_signals_{freq}.csv"
    final.to_csv(out_path, index=False)
    print(f"Saved {len(final)} rows for {len(patient_ids)} patients to {out_path}")
    return final


# ---------- usage ----------

if __name__ == "__main__":
    root = Path(r"C:\Users\Allison\OneDrive - Emory\Fan, Jason's files - CS-570-Project")
    df_all = preprocess_all(root)


C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:103: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:103: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:103: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-par

Processing patient 1023...


C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:90: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return pd.date_range(start=start, end=end, freq=freq)
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:46: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  numeric = numeric.fillna(method="ffill").fillna(method="bfill")
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:46: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  numeric = numeric.fillna(method="ffill").fillna(method="bfill")
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:60: RuntimeWarning: Mean of empty slice
  mu = np.nanmean(x)
c:\Users\Allison\anaconda3\envs\DA\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees 

Processing patient 1024...


C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:90: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return pd.date_range(start=start, end=end, freq=freq)
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:46: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  numeric = numeric.fillna(method="ffill").fillna(method="bfill")
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:46: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  numeric = numeric.fillna(method="ffill").fillna(method="bfill")
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:60: RuntimeWarning: Mean of empty slice
  mu = np.nanmean(x)
c:\Users\Allison\anaconda3\envs\DA\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees 

...

Processing patient 7408...
  Skipping 7408: No overlapping time range between CGM and HR.
Processing patient 7409...


C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:90: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return pd.date_range(start=start, end=end, freq=freq)
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:46: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  numeric = numeric.fillna(method="ffill").fillna(method="bfill")
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:46: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  numeric = numeric.fillna(method="ffill").fillna(method="bfill")
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:60: RuntimeWarning: Mean of empty slice
  mu = np.nanmean(x)
c:\Users\Allison\anaconda3\envs\DA\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees 

Processing patient 7411...


C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:90: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return pd.date_range(start=start, end=end, freq=freq)
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:46: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  numeric = numeric.fillna(method="ffill").fillna(method="bfill")
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:46: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  numeric = numeric.fillna(method="ffill").fillna(method="bfill")
C:\Users\Allison\AppData\Local\Temp\ipykernel_23436\3830587749.py:60: RuntimeWarning: Mean of empty slice
  mu = np.nanmean(x)
c:\Users\Allison\anaconda3\envs\DA\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees 

Saved 11519132 rows for 894 patients to C:\Users\Allison\OneDrive - Emory\Fan, Jason's files - CS-570-Project\preprocessed_signals_1T.csv
